<a href="https://colab.research.google.com/github/AleSSouza/desval_fipe/blob/main/consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Rode isto na primeira célula do Colab
!pip install requests pandas jinja2 openpyxl
!pip install requests pandas ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00


In [18]:
import requests
import pandas as pd
import datetime
from ipywidgets import widgets
from IPython.display import display
import time

BASE_URL = "https://parallelum.com.br/fipe/api/v1"

# Variáveis globais para cache
cached_brands = None
cached_models = {}
cached_years = {}

def get_brands():
    global cached_brands
    if cached_brands is None:
        response = requests.get(f"{BASE_URL}/carros/marcas")
        response.raise_for_status()
        cached_brands = response.json()
        time.sleep(1) # Add delay after fetching brands
    return cached_brands

def get_models(brand_id):
    global cached_models
    if brand_id not in cached_models:
        response = requests.get(f"{BASE_URL}/carros/marcas/{brand_id}/modelos")
        response.raise_for_status()
        cached_models[brand_id] = response.json().get("modelos", [])
        time.sleep(0.5)  # small pause to avoid limit
    return cached_models[brand_id]

def get_years(brand_id, model_id):
    global cached_years
    key = (brand_id, model_id)
    if key not in cached_years:
        response = requests.get(f"{BASE_URL}/carros/marcas/{brand_id}/modelos/{model_id}/anos")
        response.raise_for_status()
        cached_years[key] = response.json()
        time.sleep(0.5)
    return cached_years[key]

def get_value(brand_id, model_id, year_id):
    # Atenção: não é possível cachear tudo pois são muitas requisições, mas pode-se adicionar delay
    response = requests.get(f"{BASE_URL}/carros/marcas/{brand_id}/modelos/{model_id}/anos/{year_id}")
    response.raise_for_status()
    time.sleep(0.3)  # evitar chamadas muito rápidas
    return response.json().get("Valor")

# Widgets e funções para UI seguem igual
brand_widget = widgets.Dropdown(description="Marca:")
model_widget = widgets.Dropdown(description="Modelo:")
year_widget = widgets.Dropdown(description="Ano:")

def load_brands():
    # Add a small delay before the initial brand loading
    time.sleep(2)
    brands = get_brands()
    options = {brand["nome"]: brand["codigo"] for brand in brands}
    brand_widget.options = options

def on_brand_change(change):
    if change["type"] == "change" and change["name"] == "value":
        brand_id = change["new"]
        if brand_id:
            models = get_models(brand_id)
            options = {model["nome"]: model["codigo"] for model in models}
            model_widget.options = options
            model_widget.value = None
            year_widget.options = {}
            year_widget.value = None

def on_model_change(change):
    if change["type"] == "change" and change["name"] == "value":
        model_id = change["new"]
        brand_id = brand_widget.value
        if brand_id and model_id:
            years = get_years(brand_id, model_id)
            options = {year["nome"]: year["codigo"] for year in years}
            year_widget.options = options
            year_widget.value = None

def fetch_data(button):
    brand_id = brand_widget.value
    model_id = model_widget.value
    year_id = year_widget.value

    if not all([brand_id, model_id, year_id]):
        print("Por favor, selecione marca, modelo e ano.")
        return

    start_year = int(year_id.split("-")[0])
    today = datetime.date.today()
    if today.month > 1:
        end_year = today.year
        end_month = today.month - 1
    else:
        end_year = today.year - 1
        end_month = 12

    dados = []

    for ano in range(start_year, end_year + 1):
        for mes in range(1, 13):
            if ano == end_year and mes > end_month:
                break
            year_month_id = f"{ano}-{mes}"
            try:
                valor = get_value(brand_id, model_id, year_month_id)
            except Exception as e:
                print(f"Erro ao buscar {year_month_id}: {e}")
                valor = None
            dados.append({"Ano": ano, "Mês": mes, "Valor FIPE": valor})

    df = pd.DataFrame(dados)
    display(df)

fetch_button = widgets.Button(description="Consultar Histórico FIPE")
fetch_button.on_click(fetch_data)

brand_widget.observe(on_brand_change)
model_widget.observe(on_model_change)

load_brands()

display(brand_widget, model_widget, year_widget, fetch_button)

HTTPError: 429 Client Error: Too Many Requests for url: https://parallelum.com.br/fipe/api/v1/carros/marcas